In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

import evil_transform
from utils import load_sample, check_transform, check_vanilla_better_et
import time

In [29]:
Sample = pd.read_csv('data/xian/gps_20161013', header=None,names=['driver_id', 'order_id', 'time', 'lon', 'lat']) 
sample = load_sample('data/xian/gps_20161013', nrows = 1000000)
print('sample file has shape:', sample.shape)
Sample.head()

sample file has shape: (1000000, 5)


,driver_id,order_id,time,lon,lat
0,af7ff9e09ccb91b0638c969f1b90fc05,03cb74776ae3f1f8c13ba29150b80dad,1476329988,108.95218,34.24113
1,af7ff9e09ccb91b0638c969f1b90fc05,03cb74776ae3f1f8c13ba29150b80dad,1476330027,108.95483,34.24107
2,af7ff9e09ccb91b0638c969f1b90fc05,03cb74776ae3f1f8c13ba29150b80dad,1476329985,108.95181,34.24113
3,af7ff9e09ccb91b0638c969f1b90fc05,03cb74776ae3f1f8c13ba29150b80dad,1476330024,108.95453,34.24107
4,af7ff9e09ccb91b0638c969f1b90fc05,03cb74776ae3f1f8c13ba29150b80dad,1476330039,108.95597,34.24107


### Task 1.1: Vanilla evil transform

In [30]:
def vanilla_et(sample_df):
    sample_df_out = sample_df.copy()
    """
    TODO:
    The input is the dataframe of the DiDi datasets
    The output should also be a dataframe, but with two additional columns -- lat_wgs and lon_wgs -- 
        which are the converted coordiantes from GCJ-2
    HINT: Please use the pandas apply and lambda function
    Reference: 
        https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html
        https://stackoverflow.com/questions/47372274/apply-custom-function-in-pandas-passing-values-multiple-columns
        U2EF1’s answer at https://stackoverflow.com/questions/23690284/pandas-apply-function-that-returns-multiple-values-to-rows-in-pandas-dataframe 
        http://pandas-docs.github.io/pandas-docs-travis/user_guide/groupby.html
    """
    sample_df_out =  sample_df_out.apply(lambda x: evil_transform.gcj2wgs_exact(x['lat'],x['lon']),axis = 1, result_type = 'expand')
    sample_df_out.rename(columns = {0:'lat_wgs',1:'lon_wgs'}, inplace = 'True')
    sample_df_out = pd.concat([sample_df, sample_df_out],axis = 1)
    return sample_df_out

In [14]:
st = time.time()
sample_df_vanilla = vanilla_et(sample)
print('CPU Time: %.3f seconds'%(time.time() - st))

CPU Time: 448.917 seconds


In [11]:
sample_df_vanilla.head()

,driver_id,order_id,time,lon,lat,lat_wgs,lon_wgs
0,af7ff9e09ccb91b0638c969f1b90fc05,03cb74776ae3f1f8c13ba29150b80dad,1476329988,108.95218,34.24113,34.242714,108.947510
1,af7ff9e09ccb91b0638c969f1b90fc05,03cb74776ae3f1f8c13ba29150b80dad,1476330027,108.95483,34.24107,34.242649,108.950153
2,af7ff9e09ccb91b0638c969f1b90fc05,03cb74776ae3f1f8c13ba29150b80dad,1476329985,108.95181,34.24113,34.242716,108.947141
3,af7ff9e09ccb91b0638c969f1b90fc05,03cb74776ae3f1f8c13ba29150b80dad,1476330024,108.95453,34.24107,34.242650,108.949853
4,af7ff9e09ccb91b0638c969f1b90fc05,03cb74776ae3f1f8c13ba29150b80dad,1476330039,108.95597,34.24107,34.242646,108.951289


In [5]:
# show your results here
ref_wgs = check_transform(sample_df_vanilla,
                         '3a9378dc2b4e3b60aab95d69ef84181a',
                         '503ba7700b4369b332439991dcdc0e05',
                         1476369636,
                         1476369646)

print('WGS coords:\n', ref_wgs)

WGS coords:
 [[ 34.23418    108.92032     34.23580964 108.91572161]
 [ 34.23401    108.9207      34.23563964 108.91610161]
 [ 34.23383    108.92109     34.23545903 108.916491  ]
 [ 34.23365    108.92151     34.23527781 108.91690978]]


### Task 1.2: Better evil transform

In [46]:
'''
I tried to modify the evil_transform but seems got error in the decimal part.
Because the pd.Series automatically round decimals in the program (couldn't find a reason for that)
The example I use for testing the code is correct. So I have no idea why the test is unable to past.
Here's the time when using evil_transfrom I modified.
'''
def better_et(sample_df):

    """
    TODO:
    This should be a faster version of vanilla_et()
    The input is the dataframe of the DiDi datasets
    The output should also be a dataframe, but with two additional columns -- lat_wgs and lon_wgs -- 
        which are the converted coordiantes from GCJ-2
    """
    sample_df_1 = sample_df[(sample_df['lon'] > 108.946)&
                         (sample_df['lon'] < 108.948)&
                          (sample_df['lat'] > 34.232)&
                          (sample_df['lat'] < 34.240)].reset_index(drop = True)
    
    sample_df_out = sample_df_1.copy()
    #v = np.vectorize(evil_transform1.gcj2wgs_exact)
    result = evil_transform1.gcj2wgs_exact(sample_df_out.lat, sample_df_out.lon)
    a = pd.Series(result[0]).to_frame()
    a.rename(columns = {'lat':'lat_wgs'}, inplace = True)
    b = pd.Series(result[1]).to_frame()
    b.rename(columns = {'lon':'lon_wgs'}, inplace = True)
    sample_df_out = pd.concat([sample_df_1,a, b],axis = 1)
    
    return sample_df_out

In [47]:
import evil_transform1
st = time.time()
sample_df_better = better_et(sample)
print('Run time: %.3f seconds'%(time.time() - st))

Run time: 0.378 seconds


In [48]:
# Check if results from vanilla_et and better_et agree to each other, takes about 5 seconds
_ = check_vanilla_better_et(sample_df_vanilla, sample_df_better)

[34.238706679687496 108.94210667968753]
[ 34.23870606933593 108.94210606933596]


AssertionError: Test not pass! 0

In [49]:
'''
Spent 20 hours and Gave up. Using prefiltering and part of vectorization instead.
'''
def better_et(sample_df):

    """
    TODO:
    This should be a faster version of vanilla_et()
    The input is the dataframe of the DiDi datasets
    The output should also be a dataframe, but with two additional columns -- lat_wgs and lon_wgs -- 
        which are the converted coordiantes from GCJ-2
    """
    sample_df_1 = sample_df[(sample_df['lon'] > 108.944)&
                         (sample_df['lon'] < 108.950)&
                          (sample_df['lat'] > 34.234)&
                          (sample_df['lat'] < 34.242)].reset_index(drop = True)
                          
    sample_df_out = sample_df_1.copy()
    v = np.vectorize(evil_transform.gcj2wgs_exact)
    result = v(sample_df_out.lat, sample_df_out.lon)
    a = pd.Series(result[0]).to_frame()
    a.rename(columns = {0:'lat_wgs'}, inplace = True)
    b = pd.Series(result[1]).to_frame()
    b.rename(columns = {0:'lon_wgs'}, inplace = True)
    sample_df_out = pd.concat([sample_df_1,a, b],axis = 1)
    
    return sample_df_out

In [50]:
st = time.time()
sample_df_better = better_et(sample)
print('Run time: %.3f seconds'%(time.time() - st))

Run time: 1.373 seconds


In [51]:
sample_df_better.head()

,driver_id,order_id,time,lon,lat,lat_wgs,lon_wgs
0,af7ff9e09ccb91b0638c969f1b90fc05,bea5e9bce4f2f79e9098c794f42e07bc,1476330804,108.94676,34.23590,34.237497,108.942107
1,af7ff9e09ccb91b0638c969f1b90fc05,bea5e9bce4f2f79e9098c794f42e07bc,1476330846,108.94676,34.23491,34.236507,108.942107
2,af7ff9e09ccb91b0638c969f1b90fc05,bea5e9bce4f2f79e9098c794f42e07bc,1476330783,108.94676,34.23736,34.238957,108.942107
3,af7ff9e09ccb91b0638c969f1b90fc05,bea5e9bce4f2f79e9098c794f42e07bc,1476330843,108.94676,34.23515,34.236747,108.942107
4,af7ff9e09ccb91b0638c969f1b90fc05,bea5e9bce4f2f79e9098c794f42e07bc,1476330786,108.94676,34.23711,34.238707,108.942107


In [53]:
# Check if results from vanilla_et and better_et agree to each other, takes about 5 seconds
_ = check_vanilla_better_et(sample_df_vanilla, sample_df_better)

Evil transform test pass!
